In [9]:
# Script para encontrar Visuamente  el data drifting
# focalizado solo en los campos de un buen arbol de deicision

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

library("data.table")
library("rpart")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,619681,33.1,1321401,70.6,1321401,70.6
Vcells,1104670,8.5,8388608,64.0,1684140,12.9


Warning message:
"package 'data.table' was built under R version 4.2.3"
Warning message:
"package 'rpart' was built under R version 4.2.3"


In [ ]:
graficar_campo  <- function( campo )
{

  #quito de grafico las colas del 5% de las densidades
  qA  <- quantile(  dataset[ foto_mes==202107 , get(campo) ] , prob= c(0.05, 0.95), na.rm=TRUE )
  qB  <- quantile(  dataset[ foto_mes==202109 , get(campo) ] , prob= c(0.05, 0.95), na.rm=TRUE )

  xxmin  <- pmin( qA[[1]], qB[[1]] )
  xxmax  <- pmax( qA[[2]], qB[[2]] )

  densidad_A  <- density( dataset[ foto_mes==202107, get(campo) ],
                          kernel="gaussian", na.rm=TRUE )

  densidad_B  <- density( dataset[ foto_mes==202109, get(campo) ],
                          kernel="gaussian", na.rm=TRUE )

  plot( densidad_A,
        col="blue",
        xlim= c( xxmin, xxmax ),
        ylim= c( 0, pmax( max(densidad_A$y), max(densidad_B$y) ) ),
        main= campo 
      )

  lines(densidad_B, col="red", lty=2)
  
  legend(  "topright",  
           legend=c("202007", "202009"),
           col=c("blue", "red"), lty=c(1,2))

}


In [22]:
#Aqui comienza el programa
setwd(".\\")   #Establezco el Working Directory
getwd()


[1] "d:/Onedrive/Austral/Cursos/08 - Laboratorio de Implementacion 1/labo2023v/src/drifting"

In [23]:

#cargo el dataset donde voy a entrenar
dataset  <- fread("./datasets/dataset_pequeno.csv")

In [24]:
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/DR2930/", showWarnings = FALSE )
setwd("./exp/DR2930/")

dataset  <- dataset[ foto_mes %in% c( 202107, 202109 ) ]

#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes==202107,
         clase_binaria :=  ifelse( clase_ternaria=="CONTINUA", "NO", "SI" ) ]



In [25]:
# Entreno el modelo
# utilizo los mejores hiperparametros encontrados en una Bayesian Optimizationcon 5-fold Cross Validation
modelo  <- rpart(formula=   "clase_binaria ~ . -clase_ternaria",
                 data=      dataset[ foto_mes==202107 ],  #los datos donde voy a entrenar
                 xval=          0,
                 cp=           -0.69,
                 minsplit=    870,
                 minbucket=     9,
                 maxdepth=      9)


campos_modelo  <- names( modelo$variable.importance )
campos_buenos  <- c( campos_modelo,  setdiff( colnames(dataset), campos_modelo ) )
campos_buenos  <-  setdiff(  campos_buenos,  c( "foto_mes","clase_ternaria","clase_binaria" ) )



pdf("densidades_07_09.pdf")

for( campo in  campos_buenos )
{
  cat( campo, "  " )
  graficar_campo( campo )
}

dev.off()


ctrx_quarter   mcuentas_saldo   mcuenta_corriente   mcaja_ahorro   active_quarter   mcomisiones_mantenimiento   ccomisiones_mantenimiento   mrentabilidad_annual   mpasivos_margen   mrentabilidad   ccomisiones_otras   cdescubierto_preacordado   mcomisiones   mcomisiones_otras   ctarjeta_visa   ccajas_consultas   mactivos_margen   Master_status   mpayroll   Visa_msaldopesos   Visa_msaldototal   mtarjeta_visa_consumo   Visa_mpagominimo   cprestamos_personales   cliente_antiguedad   cpayroll_trx   ctarjeta_visa_transacciones   ctarjeta_master   mprestamos_personales   tcuentas   ccajas_otras   Visa_status   internet   ccajas_transacciones   mtransferencias_recibidas   mttarjeta_visa_debitos_automaticos   Visa_Fvencimiento   cliente_edad   numero_de_cliente   Visa_fechaalta   ctarjeta_visa_debitos_automaticos   Master_fechaalta   mcaja_ahorro_dolares   cproductos   tmobile_app   chomebanking_transacciones   Visa_mpagospesos   mtransferencias_emitidas   Master_Fvencimiento   Visa_delinquency

png 
  2